In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'



In [2]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}

--- BUild tree
tree1 = builder:build_tree(params)
tree = builder:build_tree(params)
--- build data
game = TreeData(tree)
game1 =TreeData(tree1)

In [3]:
game1:get_training_set(tree1,1)

In [4]:
--- CFR Solver
local starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
local tree_cfr = TreeCFR()
print("Solver")
tree_cfr:run_cfr(tree, starting_ranges)
print("geting training set")
game:get_training_set(tree,1)

Solver


geting training set


## Using the modules

In [6]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model

-- Building trainer:
nn_trainer = NNTrainer(game,nn_model)

-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [36]:
current_criterion = nn.AbsCriterion()

train_loss,test_loss = nn_trainer:train(nn_model,current_criterion,opt,5000)

In [37]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})

In [38]:
true_strat =nn_trainer.all_data.targets

In [39]:
strategy_nn2 = StrategyNN()
nn_model2 = strategy_nn2.model


In [40]:
nn_trainer2 = NNTrainer(game1,nn_model2)
pred_strat2 = nn_model:forward({nn_trainer2.all_data.features,nn_trainer2.all_data.masks})

In [41]:
torch.mean(torch.abs(pred_strat- true_strat))

0.21410777008471


In [42]:
torch.mean(torch.abs(nn_trainer2.all_data.targets- true_strat))

0.23573242981608


In [21]:
test_node

In [22]:
--[[
local i = 2

t2 = torch.range(1,train_loss_table[i]:size(1))
local plot = Plot()
plot:line(t2,train_loss_table[i],'red' ,'train loss')
plot:line(t2,test_loss_table[i],'blue' ,'test loss')
plot:legend(true):title('Mean Square Error')
plot:draw
--]]

 0.2405  0.2405  0.2784  0.2784  0.3039  0.3039
 0.3530  0.3530  0.2858  0.2858  0.3806  0.3806
 0.4065  0.4065  0.4357  0.4357  0.3155  0.3155
[torch.FloatTensor of size 3x6]

